In [ ]:
from experiments.config import get_default_config
from src.benchmark.synthetic_dataset import  get_task,get_dataloader
from src.libs.soi import  SOI
from src.libs.soi_grad import  SOI_grad
import numpy as np

In [ ]:
## Default config
args=get_default_config()

### Redundancy

In [ ]:
## Make a synthetic task with 6 made of two redundant subtasks of 3 variables.
my_settings = [{"rho":0.6,"type":"red","nb":3} ]
args.dim = 1
task_red = get_task(args,my_settings)
task_red.o_inf()

### Synergy

In [ ]:
## Make a synthetic task with 6 made of two redundant subtasks of 3 variables.
my_settings = [{"rho":0.6,"type":"Syn","nb":3} ]
args.dim = 1
task_syn = get_task(args,my_settings)
task_syn.get_summary()["o_inf"]

### Redundancy + Synergy

In [ ]:
## Make a synthetic task with 6 made of two redundant subtasks of 3 variables.
my_settings = [{"rho":0.6,"type":"red","nb":3},{"rho":0.6,"type":"Syn","nb":3} ]
args.dim = 1
task_both = get_task(args,my_settings)
task_both.get_summary()["o_inf"]

# O-information estimation

### General config

In [ ]:
args.use_ema=True
args.weight_s_functions = True
args.importance_sampling = True
args.bs = 512
args.warmup_epochs = 80
args.max_epochs = 60
args.test_epoch = 2
args.debug = True
args.ou_dir = "quickstart/"

### Runing SOI 

### On the redundancy benchmark

In [ ]:
gt = task_red.get_summary()
train_l, test_l  = get_dataloader(task_red,args)
## Run SOI
soi = SOI(args, nb_var = task_red.nb_var, test_loader=test_l, gt = gt)
## Fit the model
soi.fit(train_l, test_l)
## Compute O_information using the test_loader
results = soi.compute_o_inf()
print("SOI:", {" O-inf": results["o_inf"],"tc":results["tc"], "dtc":results["dtc"], "S-info": results["s_inf"] })
print("GT:", {" O-inf": gt["o_inf"],"tc":gt["tc"], "dtc":gt["dtc"], "S-info": gt["s_inf"] } )


### On the synergy benchmark

In [ ]:
gt = task_syn.get_summary()
train_l, test_l  = get_dataloader(task_syn,args)
## Run SOI
soi = SOI(args, nb_var = task_syn.nb_var, test_loader=test_l, gt = gt)
## Fit the model
soi.fit(train_l, test_l)
## Compute O_information using the test_loader
results = soi.compute_o_inf()
print("SOI:", {" O-inf": results["o_inf"],"tc":results["tc"], "dtc":results["dtc"], "S-info": results["s_inf"] })
print("GT:", {" O-inf": gt["o_inf"],"tc":gt["tc"], "dtc":gt["dtc"], "S-info": gt["s_inf"] } )


### On the mixed benchmark

In [ ]:
gt = task_both.get_summary()
train_l, test_l  = get_dataloader(task_both,args)
## Run SOI
args.warmup_epochs = 80
args.max_epochs = 100

soi = SOI(args, nb_var = task_both.nb_var, test_loader=test_l, gt = gt)
## Fit the model
soi.fit(train_l, test_l)
## Compute O_information using the test_loader
print("SOI:", {" O-inf": results["o_inf"],"tc":results["tc"], "dtc":results["dtc"], "S-info": results["s_inf"] })
print("GT:", {" O-inf": gt["o_inf"],"tc":gt["tc"], "dtc":gt["dtc"], "S-info": gt["s_inf"] } )

## Gradients of O-information estimation

In [ ]:
gt = task_both.get_summary()
print("g_o_ing : "," ".join([ "x{}: {},".format(i,np.round(x,decimals=3) ) for i,x in enumerate(gt["g_o_inf"]) ]) )

In [ ]:
## Run SOI with gradient of o-information estimation
args.warmup_epochs = 100
args.max_epochs = 120
## Run SOI
gt = task_both.get_summary()
train_l, test_l  = get_dataloader(task_both,args)
soi = SOI_grad(args, nb_var = task_both.nb_var, test_loader=test_l, gt = task_both.get_summary())
## Fit the model

soi.fit(train_l, test_l)
## Compute O_information using the test_loader
soi.compute_o_inf_with_grad(test_l)

In [ ]:
print("The final estimation :", soi.compute_o_inf_with_grad(test_l) )